## This is the version of the notebook run on Google Colab

In [1]:
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [10]:
image_dir = './images'
image_files = os.listdir(image_dir)
images = []

for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    img = load_img(image_path, target_size=(64, 64))
    img_array = img_to_array(img)
    img_array /= 255.0
    images.append(img_array)
x_train = np.array(images)
print(np.shape(img_array[0]))

(64, 3)


In [12]:
#We need a latent dimension to add to the noise data
latent_dim = 100

#Define the Generator
generator = Sequential([
    Dense(256, input_dim=latent_dim, activation='relu'),
    Dense(512, activation='relu'),
    Dense(3 * 64 * 64, activation='sigmoid'),
    Reshape((64, 64, 3))
    ]
)

# Define the Discriminator
discriminator = Sequential([
    Flatten(input_shape=(64, 64, 3)),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
    ]
)

In [13]:
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
discriminator.trainable = False

In [14]:
gan = Sequential([generator, discriminator])
gan.compile(optimizer='adam', loss='binary_crossentropy')

In [15]:
batch_size = 128
epochs = 100
steps_per_epoch = len(x_train) // batch_size

In [16]:
for epoch in range(epochs):
    for step in range(steps_per_epoch):
        # Train discriminator on real images
        real_images = x_train[np.random.randint(0, len(x_train), batch_size)] #Randomly select 'batch_size' number of images from the training data
        real_labels = np.ones((batch_size, 1)) #There are real images, set the labels for each image to 1
        d_loss_real = discriminator.train_on_batch(real_images, real_labels) # Training the discriminator so that it will be well equipped to handle the generator images

        # Generate fake images and train discriminator on them
        noise = np.random.normal(0, 1, (batch_size, latent_dim)) #We need some noise to create our fake data
        fake_images = generator.predict(noise) #Generate fake images
        fake_labels = np.zeros((batch_size, 1)) #Indicate that these are fake images by lableing each with a zero label
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels) #Training the discriminator on the fake images so that it wil know the difference

        #Now we have trained the discriminator on real and fake images. Now we train our generator

        # Train the generator (via the GAN model)
        noise = np.random.normal(0, 1, (batch_size, latent_dim)) #Noise vectors for the generated samples
        valid_labels = np.ones((batch_size, 1)) #Labels to make the fake data appear real to the discriminator
        g_loss = gan.train_on_batch(noise, valid_labels) #Treain using the generator/discriminator model

    print(f"Epoch {epoch + 1}/{epochs}, D Loss Real: {d_loss_real[0]}, D Loss Fake: {d_loss_fake[0]}, G Loss: {g_loss}")



4/4 [==============================] - 0s 3ms/step
Epoch 1/100, D Loss Real: 0.9070916175842285, D Loss Fake: 1.2453222274780273, G Loss: 2.1916329860687256
4/4 [==============================] - 0s 3ms/step
Epoch 2/100, D Loss Real: 1.256571888923645, D Loss Fake: 0.44570639729499817, G Loss: 1.1476377248764038
4/4 [==============================] - 0s 3ms/step
Epoch 3/100, D Loss Real: 0.09538257122039795, D Loss Fake: 0.14111915230751038, G Loss: 2.5067808628082275
4/4 [==============================] - 0s 3ms/step
Epoch 4/100, D Loss Real: 0.7096081972122192, D Loss Fake: 0.4659440517425537, G Loss: 1.9723408222198486
4/4 [==============================] - 0s 3ms/step
Epoch 5/100, D Loss Real: 0.42416712641716003, D Loss Fake: 0.45080626010894775, G Loss: 2.547593593597412
4/4 [==============================] - 0s 3ms/step
Epoch 6/100, D Loss Real: 0.38462182879447937, D Loss Fake: 0.36291611194610596, G Loss: 2.9635119438171387
4/4 [==============================] - 0s 3ms/step
Ep